In [4]:
import pandas as pd
import numpy as np
from joblib import load
import glob
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import zscore
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [ ]:
# Building a Classifier using Sckikit-learn


In [5]:
path = '../data/weatherdata'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

weather_df = pd.concat(li, axis=0, ignore_index=True)

/Users/danielchoy/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (43,44,48,53,57,58) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/danielchoy/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (45,46,50,55,59,88,90) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/danielchoy/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (42,43,44,45,56,57,64,90) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/danielchoy/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (42,43,44,45,49,54,58,59) have mi

In [6]:
dock = pd.read_csv('../data/stations_cleaned.csv.gz')

/Users/danielchoy/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
def to_milesperhour(num):
    """A function to convert wind speed from meters/second to miles/hour. The conversion is done by using the 
    above formula
    """
    convert = (num * 3600)/(1000 * 1.6)
    return convert

In [8]:
def clean_weatherdata(df):
    """This function will clean the weather data from any given year or years (the merged weather data)
    Weather data was obtained from NOAA( National Oceanic and Atmospheric Administration ) 
    https://www.ncei.noaa.gov/data/global-hourly/archive/csv/.
    Additional websites were also used as references in order to interpret the numbers from the data and to
    engineer new features such as windy and rainy. The column windy and its values (breeze, gale, force...)
    were all based on information from NOAA https://www.weather.gov/pqr/wind. 
    https://www.visualcrossing.com/resources/documentation/weather-data/how-we-process-integrated-surface-database-historical-weather-data/
    was used to interpret the numbers of the data.
    """
    # Make a copy
    nyweather= df.copy()
    
    # Select the columns that will be used
    nyweather = nyweather[['DATE', 'SOURCE', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'NAME', 'WND', 'TMP',
             'DEW', 'SLP', 'AA1','AA2']]
    
    # Clean DATE column
    nyweather['DATE'] = nyweather['DATE'].str.replace('T', ' ')
    
    # Clean TMP (temperature) column. Please refer to 
    # Interpreting the numbers are based on 
    #https://www.visualcrossing.com/resources/documentation/weather-data/how-we-process-integrated-surface-database-historical-weather-data/

    nyweather['TMP'] = nyweather['TMP'].astype(str)
    nyweather['TMP'] = nyweather['TMP'].str.slice(0, -2)
    nyweather['TMP'] = nyweather['TMP'].str.replace('+','')
    nyweather = nyweather.drop(nyweather[nyweather.TMP == '9999'].index)
    nyweather['TMP'] = nyweather['TMP'].astype(int)
    nyweather['TMP'] = nyweather['TMP']/10
    print('complete part 1')
    
    # Clean WND column
    # Please refer to https://www.weather.gov/pqr/wind
    nyweather['WND'] = nyweather['WND'].astype(str)
    nyweather = pd.concat([nyweather, nyweather.WND.str.split(',', expand = True)],1)
    nyweather = nyweather.rename(columns={3:'wind_speed'})
    nyweather['wind_speed'] = nyweather['wind_speed'].astype(int)
    nyweather['wind_speed'] = nyweather['wind_speed']/10
    nyweather['wind_speed'] = nyweather['wind_speed'].map(to_milesperhour)
    
    # Engineered a new feature called windy
    nyweather['windy'] = nyweather['wind_speed'].apply(lambda x: 'calm' if x < 4 else 'breeze' if x < 12 else \
                                        'moderate breeze' if x < 24 else 'strong breeze' if x < 31 else \
                                        'gale' if x < 63 else 'storm force')
    nyweather = nyweather[['DATE', 'SOURCE', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'NAME', 'WND', 'TMP',\
                           'DEW', 'SLP', 'AA1','AA2', 'wind_speed', 'windy']]
    
    print('complete part 2')
    
    # Clean the AA1 column that has precipitation information
    nyweather['AA1'] = nyweather['AA1'].fillna('0,0,0,0')
    nyweather['AA1'] = nyweather['AA1'].astype(str)
    nyweather = pd.concat([nyweather, nyweather.AA1.str.split(',', expand = True)],1)
    nyweather = nyweather.rename(columns={1:'precipitation'})
    nyweather['precipitation'] = nyweather['precipitation'].astype(int)
    nyweather['precipitation'] = nyweather['precipitation']/10
    nyweather = nyweather[['DATE', 'SOURCE', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'NAME', 'WND', 'TMP',\
                           'DEW', 'SLP', 'wind_speed', 'windy', 'precipitation']]
    
    # Engineered a new feature called rainy
    nyweather['rainy'] = nyweather['precipitation'].apply(lambda x: 'rainy' if x > 0 else 'not rainy')
    
    
    # Clean DATE column
    # Generate new features - month, date, hour and month2
    nyweather['DATE'] = nyweather['DATE'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    nyweather['month'] = nyweather.DATE.dt.month
    nyweather['date'] = nyweather.DATE.dt.date
    nyweather['hour'] = nyweather.DATE.dt.hour
    nyweather['month2'] = nyweather.DATE.dt.month.apply(lambda x: 'Jan' if x == 1 else 'Feb' if x == 2 else \
    'Mar' if x == 3 else 'Apr' if x == 4 else 'May' if x == 5 else 'Jun' if x == 6 else 'Jul' if x == 7 else \
    'Aug' if x == 8 else 'Sep' if x == 9 else 'Oct' if x == 10 else 'Nov' if x == 11 else 'Dec' )
    
    # Generate cleaned dataframe
    return nyweather

In [9]:
weather_df = clean_weatherdata(weather_df)

complete part 1
complete part 2


In [25]:
dock

,dock_id,dock_name,date,hour,minute,avail_bikes,avail_docks,tot_docks,_lat,_long,in_service,status_key,dock_status,time,dayofweek,dayofweek2,season
0,72,W 52 St & 11 Ave,2015-03-01,1,6,8,31,39,40.767272,-73.993929,1,1,Empty Alert,1:6,6,Sunday,spring
1,72,W 52 St & 11 Ave,2015-03-01,1,16,8,31,39,40.767272,-73.993929,1,1,Empty Alert,1:16,6,Sunday,spring
2,72,W 52 St & 11 Ave,2015-03-01,1,25,8,30,38,40.767272,-73.993929,1,1,Empty Alert,1:25,6,Sunday,spring
3,72,W 52 St & 11 Ave,2015-03-01,1,35,8,30,38,40.767272,-73.993929,1,1,Empty Alert,1:35,6,Sunday,spring
4,72,W 52 St & 11 Ave,2015-03-01,1,45,8,30,38,40.767272,-73.993929,1,1,Empty Alert,1:45,6,Sunday,spring
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32391416,3534,Frederick Douglass Blvd & W 117 St,2019-01-31,21,28,18,21,39,40.805159,-73.954692,1,1,Healthy,21:28,3,Thursday,winter
32391417,3534,Frederick Douglass Blvd & W 117 St,2019-01-31,22,7,18,21,39,40.805159,-73.954692,1,1,Healthy,22:7,3,Thursday,winter
32391418,3534,Frederick Douglass Blvd & W 117 St,2019-01-31,22,44,18,21,39,40.805159,-73.954692,1,1,Healthy,22:44,3,Thursday,winter
32391419,3534,Frederick Douglass Blvd & W 117 St,2019-01-31,23,32,19,20,39,40.805159,-73.954692,1,1,Healthy,23:32,3,Thursday,winter


In [12]:
daily_temp = weather_df.groupby(["date", "hour"]).agg({
  "TMP" : ['min', 'max', 'mean'], 
  "wind_speed" : ["min", 'max', 'mean'], 
  "precipitation" : ["sum"]
  }).reset_index()
daily_temp.columns = ["_".join(x) for x in daily_temp.columns.ravel()]


In [13]:
daily_temp

,date_,hour_,TMP_min,TMP_max,TMP_mean,wind_speed_min,wind_speed_max,wind_speed_mean,precipitation_sum
0,2013-01-01,0,3.3,3.9,3.6,13.950,17.325,15.6375,0.0
1,2013-01-01,1,3.9,3.9,3.9,13.950,13.950,13.9500,0.0
2,2013-01-01,2,4.4,4.4,4.4,15.075,15.075,15.0750,0.0
3,2013-01-01,3,4.4,4.4,4.4,15.075,16.200,15.6375,0.0
4,2013-01-01,4,5.0,5.0,5.0,18.450,18.450,18.4500,0.0
...,...,...,...,...,...,...,...,...,...
69602,2020-12-10,3,3.9,3.9,3.9,13.950,13.950,13.9500,0.0
69603,2020-12-10,4,4.4,4.4,4.4,10.350,10.350,10.3500,0.0
69604,2020-12-10,5,3.9,3.9,3.9,8.100,8.100,8.1000,0.0
69605,2020-12-10,6,4.4,4.4,4.4,10.350,10.350,10.3500,0.0


In [14]:
daily_temp = daily_temp.rename(columns = {'date_': 'date', 'hour_':'hour'})

In [19]:
daily_temp['date'] = pd.to_datetime(daily_temp['date'])

In [22]:
dock['date'] = pd.to_datetime(dock['date'])

In [23]:
combined = dock.merge(daily_temp, on = ['date','hour'], how = 'inner')

In [28]:
combined.dtypes

dock_id                       int64
dock_name                    object
date                 datetime64[ns]
hour                          int64
minute                        int64
avail_bikes                   int64
avail_docks                   int64
tot_docks                     int64
_lat                        float64
_long                       float64
in_service                    int64
status_key                   object
dock_status                  object
time                         object
dayofweek                     int64
dayofweek2                   object
season                       object
TMP_min                     float64
TMP_max                     float64
TMP_mean                    float64
wind_speed_min              float64
wind_speed_max              float64
wind_speed_mean             float64
precipitation_sum           float64
dtype: object

In [26]:
pd.set_option('display.max_columns', None)

In [30]:
combined['min'] = combined['minute'].apply(lambda x: '00' if x < 20 else '20' if x < 40 else '40')

In [33]:
combined['min'] = combined['min'].astype(int)

In [36]:
combined['rainy'] = combined['precipitation_sum'].apply(lambda x: 'rainy' if x > 0 else 'not rainy')

In [37]:
combined['windy'] = combined['wind_speed_mean'].apply(lambda x: 'calm' if x < 4 else 'breeze' if x < 12 else \
                                        'moderate breeze' if x < 24 else 'strong breeze' if x < 31 else \
                                        'gale' if x < 63 else 'storm force')

In [45]:
combined['season2'] = combined['season'].apply(lambda x: 1 if x == 'spring' else 2 if x == 'summer' else \
                                             3 if x == 'fall' else 4)

In [47]:
combined

,dock_id,dock_name,date,hour,minute,avail_bikes,avail_docks,tot_docks,_lat,_long,in_service,status_key,dock_status,time,dayofweek,dayofweek2,season,TMP_min,TMP_max,TMP_mean,wind_speed_min,wind_speed_max,wind_speed_mean,precipitation_sum,min,rainy,windy,season2
0,72,W 52 St & 11 Ave,2015-03-01,1,6,8,31,39,40.767272,-73.993929,1,1,Empty Alert,1:6,6,Sunday,spring,-2.8,-2.8,-2.8,5.85,5.850,5.8500,0.0,0,not rainy,breeze,1
1,72,W 52 St & 11 Ave,2015-03-01,1,16,8,31,39,40.767272,-73.993929,1,1,Empty Alert,1:16,6,Sunday,spring,-2.8,-2.8,-2.8,5.85,5.850,5.8500,0.0,0,not rainy,breeze,1
2,72,W 52 St & 11 Ave,2015-03-01,1,25,8,30,38,40.767272,-73.993929,1,1,Empty Alert,1:25,6,Sunday,spring,-2.8,-2.8,-2.8,5.85,5.850,5.8500,0.0,20,not rainy,breeze,1
3,72,W 52 St & 11 Ave,2015-03-01,1,35,8,30,38,40.767272,-73.993929,1,1,Empty Alert,1:35,6,Sunday,spring,-2.8,-2.8,-2.8,5.85,5.850,5.8500,0.0,20,not rainy,breeze,1
4,72,W 52 St & 11 Ave,2015-03-01,1,45,8,30,38,40.767272,-73.993929,1,1,Empty Alert,1:45,6,Sunday,spring,-2.8,-2.8,-2.8,5.85,5.850,5.8500,0.0,40,not rainy,breeze,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31033365,3530,Crescent St & Ditmars Blvd,2019-01-22,6,17,1,17,19,40.778718,-73.914564,1,1,Empty Alert,6:17,1,Tuesday,winter,-10.0,-9.4,-9.7,18.45,23.175,20.8125,0.0,0,not rainy,moderate breeze,4
31033366,3531,Frederick Douglass Blvd & W 129 St,2019-01-22,6,17,3,24,27,40.812551,-73.949229,1,1,Empty Alert,6:17,1,Tuesday,winter,-10.0,-9.4,-9.7,18.45,23.175,20.8125,0.0,0,not rainy,moderate breeze,4
31033367,3532,Ditmars Blvd & 19 St,2019-01-22,6,17,2,25,27,40.781450,-73.918330,1,1,Empty Alert,6:17,1,Tuesday,winter,-10.0,-9.4,-9.7,18.45,23.175,20.8125,0.0,0,not rainy,moderate breeze,4
31033368,3533,St. Nicholas Ave & W 126 St,2019-01-22,6,17,12,22,34,40.811432,-73.951878,1,1,Healthy,6:17,1,Tuesday,winter,-10.0,-9.4,-9.7,18.45,23.175,20.8125,0.0,0,not rainy,moderate breeze,4


In [48]:
X = combined[['dock_id', 'hour', 'min', 'season2', 'dayofweek', 'TMP_mean', 'precipitation_sum','wind_speed_mean']]

In [49]:
y = combined['dock_status']

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) # 80% training and 20% test

In [51]:
clf=RandomForestClassifier(n_estimators=100)

In [ ]:
clf.fit(X_train,y_train)

In [ ]:
y_pred=clf.predict(X_test)

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))